The Llama 2 is a collection of pretrained and fine-tuned generative text models, ranging from 7 billion to 70 billion parameters, designed for dialogue use cases.

 It outperforms open-source chat models on most benchmarks and is on par with popular closed-source models in human evaluations for helpfulness and safety.

In [1]:
!nvidia-smi

Tue Feb  4 08:26:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python numpy
!pip install huggingface_hub
!pip install llama-cpp-python
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llama-cpp-python
Failed to build llama-cpp-python
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (llama-cpp-python)
  Using cached llama_cpp_python-0.3.7.tar.gz (66.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pypr

In [11]:
from huggingface_hub import hf_hub_download

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"  # Use GGUF model
model_basename = "llama-2-13b-chat.Q5_K_M.gguf"        # Ensure GGUF format

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)


llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [12]:
import os
print(f"Model Path: {model_path}")
print(f"File Exists: {os.path.isfile(model_path)}")

Model Path: /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf
File Exists: True


## ***Import All the Required Libraries***

In [13]:
from huggingface_hub import hf_hub_download

In [14]:
from llama_cpp import Llama

## ***3 Download the Model***

In [15]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [16]:
model_path

'/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf'

## ***Loading the Model***

In [17]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_

## ***Create a Prompt Template***

In [27]:
prompt = "Write a linear regression code"
prompt_template = f'''
### SYSTEM:
You are a helpful, respectful, and honest assistant. Provide concise and accurate code when asked.

### USER:
{prompt}

### ASSISTANT:
'''

# Remove echo=True and adjust parameters
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=512,         # Increased token limit
    temperature=0.2,        # Lower temperature for deterministic output
    top_p=0.9,
    repeat_penalty=1.1,
    top_k=50
)

# Access the generated text
print(response["choices"][0]["text"].strip())

Llama.generate: 1 prefix-match hit, remaining 54 prompt tokens to eval
llama_perf_context_print:        load time =   34865.12 ms
llama_perf_context_print: prompt eval time =   48501.57 ms /    54 tokens (  898.18 ms per token,     1.11 tokens per second)
llama_perf_context_print:        eval time =  434914.01 ms /   368 runs   ( 1181.83 ms per token,     0.85 tokens per second)
llama_perf_context_print:       total time =  483941.55 ms /   422 tokens


Of course! Here is a linear regression code in Python using scikit-learn library:
```
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv('data.csv')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Create linear regression object and fit the data
reg = LinearRegression()
reg.fit(X_train, y_train)

# Make predictions on testing set
y_pred = reg.predict(X_test)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean squared error: {mse:.2f}')
print(f'R-squared: {r2:.2f}')
```
This code assumes that you have a CSV file named `data.csv` containing the data, with the target variable in the last column and the other features in the previous columns. The code splits the data into training and testing sets, creates

In [28]:
# Access the generated text
print(response["choices"][0]["text"].strip())

Of course! Here is a linear regression code in Python using scikit-learn library:
```
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv('data.csv')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Create linear regression object and fit the data
reg = LinearRegression()
reg.fit(X_train, y_train)

# Make predictions on testing set
y_pred = reg.predict(X_test)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean squared error: {mse:.2f}')
print(f'R-squared: {r2:.2f}')
```
This code assumes that you have a CSV file named `data.csv` containing the data, with the target variable in the last column and the other features in the previous columns. The code splits the data into training and testing sets, creates